# llamaindex agents

In [40]:
!pip install openai
!pip install python-dotenv
!pip install llama-index

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


# Quick start with Llamaindex Agents

### Based on LLamaindex documentation: [click](https://docs.llamaindex.ai/en/stable/understanding/agent/basic_agent/)

In [41]:
import os
import json
import requests
from dotenv import dotenv_values

In [42]:
secrets = dotenv_values(".env")
os.environ["OPENAI_API_KEY"] = secrets["OPENAI_API_KEY"]

### Tools creation

In [43]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

def subtract(a: int, b: int) -> int:
    """Subtract two integers and returns the result integer"""
    return a - b

In [44]:
from llama_index.core.tools import FunctionTool

# Define a tools from functions

multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
subtract_tool = FunctionTool.from_defaults(fn=subtract)

# Create a list of tools to be used by agent
tools = [multiply_tool, add_tool, subtract_tool]

### Create an Agent

In [45]:
from llama_index.llms.openai import OpenAI
from llama_index.agent.openai import OpenAIAgent

# Create LLM client
llm = OpenAI(model="gpt-4o")

# Create OpenAIAgent 
agent = OpenAIAgent.from_tools(tools, llm=llm, verbose=True)

In [51]:
output = agent.chat("What is the result: 20 + 4 * 2")

Added user message to memory: What is the result: 20 + 4 * 2
=== Calling Function ===
Calling function: multiply with args: {"a": 4, "b": 2}
Got output: 8

=== Calling Function ===
Calling function: add with args: {"a": 20, "b": 8}
Got output: 28



In [52]:
output.response

'The result of the expression \\(20 + 4 \\times 2\\) is \\(28\\).'

In [ ]:
### TODO Add weather checking logic

In [46]:
agent.chat("What is the weather in Stockholm?")

Added user message to memory: What is the weather in Stockholm?


AgentChatResponse(response="I currently don't have access to real-time data, including weather updates. You can check the latest weather information for Stockholm on a weather website or app.", sources=[], source_nodes=[], is_dummy_stream=False, metadata=None)

In [47]:
agent.chat("What is the weather sunnier in Stockholm than in Paris?")

Added user message to memory: What is the weather sunnier in Stockholm than in Paris?


AgentChatResponse(response="I currently don't have access to real-time weather data to compare the weather conditions in Stockholm and Paris. For the most accurate and up-to-date weather information, please check a reliable weather website or app for both cities.", sources=[], source_nodes=[], is_dummy_stream=False, metadata=None)

In [48]:
agent.chat("Give me the weather in 3 europe largest cities?")

Added user message to memory: Give me the weather in 3 europe largest cities?


AgentChatResponse(response="I currently don't have access to real-time weather data. For the most accurate and up-to-date weather information, please check a reliable weather website or app for the largest cities in Europe, such as Moscow, London, and Berlin.", sources=[], source_nodes=[], is_dummy_stream=False, metadata=None)

# Deep dive into Llamaindex API

### Tool creation with High Level API

In [4]:
from typing import Any, Type
from typing import cast
from llama_index.core.types import Model
from llama_index.core.bridge.pydantic import BaseModel

## High level FunctionTool API
def get_weather(city: str):
    """Calls open weather api to get weather in given city"""
    url = """https://api.openweathermap.org/data/2.5/weather?q={}&appid={}&units=metric""".format(city, secrets["OPEN_WEATHER_API_KEY"])
    response = requests.get(url)

    if response.status_code != 200:
        raise Exception("An error has occured, code: {}, reason: {}".format(response.status_code, response.reason))
    data_dict = response.json()

    return json.dumps({
        "city": data_dict['name'],
        "weather": data_dict['weather'][0]['description'],
        "temperature": data_dict['main']['temp'],
        "unit": "Celsius"
    })

In [5]:
weather_tool = FunctionTool.from_defaults(fn=get_weather)

### Tool created with Low Level API

In [6]:
from typing import Any, Type

from llama_index.core.types import Model
from llama_index.core.tools import ToolMetadata, ToolOutput
from llama_index.core.tools.types import BaseTool
from llama_index.core.bridge.pydantic import BaseModel

# Low level BaseTool API
class PydanticOutputTool(BaseTool):

    @staticmethod
    def mk_metadata(output_cls) -> ToolMetadata:
        name = "pydantic_output_tool"
        description = (
            "Use this tool to provide structured output"
        )

        return ToolMetadata(
            name=name,
            description=description,
            fn_schema=output_cls
        )

    def __init__(self, output_cls: Type[Model]):
        self._output_cls = output_cls

        fn_schema = output_cls,
        def model_fn(**kwargs: Any) -> BaseModel:
            """Model function."""
            return self._output_cls(**kwargs)
        self._fn = model_fn
        self._metadata = self.mk_metadata(output_cls=self._output_cls)


    def __call__(self, *args: Any, **kwargs: Any) -> ToolOutput:
        tool_output = self._fn(**kwargs)

        return ToolOutput(
            content=str(tool_output),
            tool_name=self.metadata.name,
            raw_input={"args": args, "kwargs": kwargs},
            raw_output=tool_output,
        )

    @property
    def metadata(self) -> ToolMetadata:
        """Metadata."""
        return self._metadata

## Based on: https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/program/function_program.py

In [7]:
# Define container for weather data
class WeatherDetails(BaseModel):
    value: str
    unit: str
    description: str
    city: str

# Init tool for making data structured
pydantic_output_tool = PydanticOutputTool(WeatherDetails)

### Create an Agent

In [10]:
from llama_index.llms.openai import OpenAI
from llama_index.agent.openai import OpenAIAgent

# Create LLM client
llm = OpenAI(model="gpt-4o")

# Create a list of tools to be used by agent
tools = [weather_tool, pydantic_output_tool]

# Create OpenAIAgent 
agent = OpenAIAgent.from_tools(tools, llm=llm, verbose=True)

### Call an Agent

In [28]:
output = agent.chat("What is the weather is Paris? Please provide structured response")

Added user message to memory: What is the weather is Paris? Please provide structured response
=== Calling Function ===
Calling function: get_weather with args: {"city":"Paris"}
Got output: {"city": "Paris", "weather": "scattered clouds", "temperature": 14.01, "unit": "Celsius"}

=== Calling Function ===
Calling function: pydantic_output_tool with args: {"value":"14.01","unit":"Celsius","description":"scattered clouds","city":"Paris"}
Got output: value='14.01' unit='Celsius' description='scattered clouds' city='Paris'



### Examine the output

In [29]:
output

AgentChatResponse(response='The weather in Paris is currently 14.01°C with scattered clouds.', sources=[ToolOutput(content='{"city": "Paris", "weather": "scattered clouds", "temperature": 14.01, "unit": "Celsius"}', tool_name='get_weather', raw_input={'args': (), 'kwargs': {'city': 'Paris'}}, raw_output='{"city": "Paris", "weather": "scattered clouds", "temperature": 14.01, "unit": "Celsius"}', is_error=False), ToolOutput(content="value='14.01' unit='Celsius' description='scattered clouds' city='Paris'", tool_name='pydantic_output_tool', raw_input={'args': (), 'kwargs': {'value': '14.01', 'unit': 'Celsius', 'description': 'scattered clouds', 'city': 'Paris'}}, raw_output=WeatherDetails(value='14.01', unit='Celsius', description='scattered clouds', city='Paris'), is_error=False)], source_nodes=[], is_dummy_stream=False, metadata=None)

In [12]:
output.response

'The weather in Paris is currently 14.01°C with scattered clouds.'

In [23]:
output.sources

[ToolOutput(content='{"city": "Paris", "weather": "scattered clouds", "temperature": 14.01, "unit": "Celsius"}', tool_name='get_weather', raw_input={'args': (), 'kwargs': {'city': 'Paris'}}, raw_output='{"city": "Paris", "weather": "scattered clouds", "temperature": 14.01, "unit": "Celsius"}', is_error=False),
 ToolOutput(content="value='14.01' unit='Celsius' description='scattered clouds' city='Paris'", tool_name='pydantic_output_tool', raw_input={'args': (), 'kwargs': {'value': '14.01', 'unit': 'Celsius', 'description': 'scattered clouds', 'city': 'Paris'}}, raw_output=WeatherDetails(value='14.01', unit='Celsius', description='scattered clouds', city='Paris'), is_error=False)]

In [27]:
output.sources[-1].raw_output

WeatherDetails(value='14.01', unit='Celsius', description='scattered clouds', city='Paris')

### Others

In [33]:
def get_stock_price(symbol: str):
    url = """https://api.polygon.io/v2/aggs/ticker/{}/prev?adjusted=true&apiKey={}""".format(symbol, secrets["POLYGLON_API_KEY"])
    response = requests.get(url)

    if response.status_code != 200:
        raise Exception("An error has occured, code: {}, reason: {}".format(response.status_code, response.reason))
    data_dict = response.json()
    return json.dumps({
        "ticker": data_dict['results'][0]['T'],
        "price": data_dict['results'][0]['c']
    })
    return data_dict['results'][0]['T'], data_dict['results'][0]['c']